# Mapy interaktywne

***Autor: Adam Dąbkowski***

## 0. Importowanie niezbędnych bibliotek

In [1]:
import pandas as pd
import plotly.express as px

from ipywidgets import interact, widgets

## 1. Wczytywanie zbioru danych

Źródło danych: ***https://www.kaggle.com/datasets/ujwalkandi/unesco-world-heritage-sites/data***

In [2]:
DATASET_PATH: str = "whc-sites-2019.csv"

In [3]:
df = pd.read_csv(DATASET_PATH, low_memory=False)
df.head(n=100)

,category,states_name_en,region_en,unique_number,id_no,rev_bis,name_en,short_description_en,justification_en,date_inscribed,...,date_end,danger_list,longitude,latitude,area_hectares,criteria_txt,category_short,iso_code,udnp_code,transboundary
0,Cultural,Afghanistan,Asia and the Pacific,230,208,Rev,Cultural Landscape and Archaeological Remains ...,The cultural landscape and archaeological rem...,<em>Criterion (i):</em> The Buddha statues an...,2003,...,NaN,Y 2003,67.825250,34.846940,1.589265e+02,(i)(ii)(iii)(iv)(vi),C,af,afg,0
1,Cultural,Afghanistan,Asia and the Pacific,234,211,Rev,Minaret and Archaeological Remains of Jam,"The 65m-tall Minaret of Jam is a graceful, so...",<em>Criterion (ii):</em> The innovative archi...,2002,...,NaN,Y 2002,64.515889,34.396417,7.000000e+01,(ii)(iii)(iv),C,af,afg,0
2,Cultural,Albania,Europe and North America,1590,569,Bis,Historic Centres of Berat and Gjirokastra,Berat and Gjirokastra are inscribed as rare e...,NaN,2005,...,NaN,NaN,20.133333,40.069444,5.890000e+01,(iii)(iv),C,al,alb,0
3,Cultural,Albania,Europe and North America,1563,570,ter,Butrint,"Inhabited since prehistoric times, Butrint ha...",NaN,1992,...,2005.0,P 1997-2005,20.026111,39.751111,NaN,(iii),C,al,alb,0
4,Cultural,Algeria,Arab States,111,102,NaN,Al Qal'a of Beni Hammad,In a mountainous site of extraordinary beauty...,NaN,1980,...,NaN,NaN,4.786840,35.818440,1.500000e+02,(iii),C,dz,dza,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Natural,Brazil,Latin America and the Caribbean,1044,892,Rev,Discovery Coast Atlantic Forest Reserves,"The Discovery Coast Atlantic Forest Reserves,...",<em>Criterion (ix):</em> The Brazilian Discov...,1999,...,NaN,NaN,-39.250000,-16.500000,1.119300e+05,(ix)(x),N,br,bra,0
96,Natural,Brazil,Latin America and the Caribbean,1045,893,-894 Rev,Atlantic Forest South-East Reserves,"The Atlantic Forest South-East Reserves, in t...",The Atlantic Forests (Southeast) contain the ...,1999,...,NaN,NaN,-48.000000,-24.166670,4.681930e+05,(vii)(ix)(x),N,br,bra,0
97,Cultural,Brazil,Latin America and the Caribbean,1157,993,Rev,Historic Centre of the Town of Goiás,Goiás testifies to the occupation and coloniz...,<em>Criterion ii:</em> In its layout and arch...,2001,...,NaN,NaN,-50.133360,-15.933280,4.030000e+01,(ii)(iv),C,br,bra,0
98,Natural,Brazil,Latin America and the Caribbean,1163,998,Bis,Central Amazon Conservation Complex,The Central Amazon Conservation Complex makes...,<em>Criterion (ix):</em> The varzea and igap&...,2000,...,NaN,NaN,-62.008333,-2.333333,5.323018e+06,(ix)(x),N,br,bra,0


## 2. Wizualizacja danych

#### 2.1 Implementacja funkcji pomocniczych

In [4]:
def wrap_text(text, width=50):
    if not isinstance(text, str):
        return text
    
    words = text.split()
    wrapped_lines = []
    current_line = []
    current_length = 0

    for word in words:
        if current_length + len(word) + 1 > width:
            wrapped_lines.append(' '.join(current_line))
            current_line = [word]
            current_length = len(word) + 1
        else:
            current_line.append(word)
            current_length += len(word) + 1

    if current_line:
        wrapped_lines.append(' '.join(current_line))

    return '<br>'.join(wrapped_lines)

#### 2.2 Generacja interaktywnej mapy

In [5]:
df['date_inscribed'] = pd.to_numeric(df['date_inscribed'], errors='coerce')

regions = ['All'] + df['region_en'].dropna().unique().tolist()
countries = ['All'] + df['states_name_en'].dropna().unique().tolist()

global_min_year = 1978
global_max_year = int(df['date_inscribed'].max())

region_widget = widgets.Dropdown(
    options=regions,
    value='All',
    description='Region:'
)

country_widget = widgets.Dropdown(
    options=countries,
    value='All',
    description='Country:'
)

min_year_widget = widgets.IntSlider(
    value=global_min_year,
    min=global_min_year,
    max=global_max_year,
    step=1,
    description='Min Year:',
    continuous_update=False
)

max_year_widget = widgets.IntSlider(
    value=global_max_year,
    min=global_min_year,
    max=global_max_year,
    step=1,
    description='Max Year:',
    continuous_update=False
)

def update_countries(region):
    if region == 'All':
        countries = ['All'] + df['states_name_en'].dropna().unique().tolist()
    else:
        countries = ['All'] + df[df['region_en'] == region]['states_name_en'].dropna().unique().tolist()
    country_widget.options = countries

region_widget.observe(lambda change: update_countries(change.new), names='value')

def update_map(region='All', country='All', min_year=global_min_year, max_year=global_max_year):
    if region == 'All':
        filtered_data = df
    else:
        filtered_data = df[df['region_en'] == region]

    if country != 'All':
        filtered_data = filtered_data[filtered_data['states_name_en'] == country]

    filtered_data = filtered_data[
        (filtered_data['date_inscribed'] >= min_year) &
        (filtered_data['date_inscribed'] <= max_year)
    ]

    filtered_data = filtered_data.copy()

    if filtered_data.empty:
        fig = px.scatter_geo(
            data_frame=None,
            lat=None,
            lon=None,
            title="UNESCO World Heritage Sites - No data available for the selected filters",
            projection="natural earth"
        )
    else:
        fig = px.scatter_geo(
            filtered_data,
            lat='latitude',
            lon='longitude',
            hover_name='name_en',
            hover_data=['category', 'region_en', 'date_inscribed', 'short_description_en'],
            color='category',
            title=f"UNESCO World Heritage Sites - {country if country != 'All' else (region if region != 'All' else 'All Regions')} ({min_year}-{max_year})",
            projection="natural earth"
        )
        
        filtered_data['wrapped_description'] = filtered_data['short_description_en'].apply(lambda x: wrap_text(x))
        
        fig.update_traces(
            hovertemplate=(
                "<b>%{hovertext}</b><br><br>" +
                "Category: %{customdata[0]}<br>" +
                "Region: %{customdata[1]}<br>" +
                "Year Inscribed: %{customdata[2]}<br><br>" +
                "Description:<br>%{customdata[3]}<extra></extra>"
            ),
            customdata=filtered_data[['category', 'region_en', 'date_inscribed', 'wrapped_description']].values,
            hovertext=filtered_data['name_en']
        )

    fig.update_layout(
        geo=dict(
            projection_scale=1.4,
            showland=True,
            landcolor="lightgray",
            showcountries=True,
            countrycolor="darkgray",
            showocean=True,
            oceancolor="azure",
            showframe=False,
        ),
        width=1110,
        height=650,
        margin={"r": 0, "t": 40, "l": 0, "b": 0},
        title={
            'y': 0.95,
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top',
            'font': {'size': 20, 'color': 'black'}
        }
    )

    fig.show()

interact(update_map, region=region_widget, country=country_widget, min_year=min_year_widget, max_year=max_year_widget)

interactive(children=(Dropdown(description='Region:', options=('All', 'Asia and the Pacific', 'Europe and Nort…

<function __main__.update_map(region='All', country='All', min_year=1978, max_year=2019)>